In [32]:
from pathlib import Path
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
import plotly.express as px
import umap
import gensim
import warnings
warnings.filterwarnings("ignore")

In [33]:
BASE_DIR = Path.cwd().parent
PALETTE = px.colors.qualitative.Prism

In [34]:
df_hosts_reviews_en = pd.read_parquet(BASE_DIR / 'processed_data' / 'hosts_reviews_en_topics.parquet')

In [35]:
df_texts = df_hosts_reviews_en.query('(comments_language == "en") & (description_language == "en")')

In [36]:
# define a function to calculate Jaccard similarity
def jaccard_similarity(col_1, col_2):
    tokens_1 = set(word_tokenize(col_1))
    tokens_2 = set(word_tokenize(col_2))
    intersection = tokens_1.intersection(tokens_2)
    union = tokens_1.union(tokens_2)
    return len(intersection) / len(union)


# apply the Jaccard similarity function to each pair of text strings
df_texts['jaccard_similarity'] = df_texts.apply(lambda row: jaccard_similarity(row['comments'], row['host_description']), axis=1)


In [37]:
# define a function to calculate cosine similarity
def calculate_cosine_similarity(col_1, col_2):
    tfidf_vectorizer = TfidfVectorizer()
    tfidf_matrix = tfidf_vectorizer.fit_transform([col_1, col_2])
    return cosine_similarity(tfidf_matrix)[0][1]

# apply the cosine similarity function to each pair of text strings
df_texts['cosine_similarity'] = df_texts.apply(lambda row: calculate_cosine_similarity(row['comments'], row['host_description']), axis=1)

In [38]:
df_texts[['jaccard_similarity', 'cosine_similarity']].mean()

jaccard_similarity    0.033041
cosine_similarity     0.050504
dtype: float64

In [39]:
df_texts[['jaccard_similarity', 'cosine_similarity']].max()

jaccard_similarity    0.300000
cosine_similarity     0.488891
dtype: float64

In [40]:
cosine_sim_by_neighbourhood = (df_texts.groupby(['listing_id_encod', 'neighbourhood', 
                                                 'review_scores_value', 'price',
                                                 'dominant_topic'])
        .agg(number_of_reviews=('number_of_reviews', 'count'), cosine_similarity=('cosine_similarity', 'mean'))
        .sort_values('cosine_similarity', ascending=False)
        .reset_index())
cosine_sim_by_neighbourhood

,listing_id_encod,neighbourhood,review_scores_value,price,dominant_topic,number_of_reviews,cosine_similarity
0,2534.0,Hässelby-Vällingby,3.00,550.0,1,1,0.336677
1,436.0,Skarpnäcks,5.00,2500.0,2,2,0.250108
2,436.0,Skarpnäcks,5.00,2500.0,1,2,0.230351
3,414.0,Spånga-Tensta,4.64,2824.0,1,2,0.229463
4,2102.0,Bromma,5.00,1195.0,1,1,0.224910
...,...,...,...,...,...,...,...
4770,400.0,Kungsholmens,4.63,900.0,2,2,0.000000
4771,2572.0,Bromma,4.50,900.0,1,1,0.000000
4772,400.0,Kungsholmens,4.63,900.0,1,1,0.000000
4773,400.0,Kungsholmens,4.63,900.0,0,2,0.000000


In [41]:
px.density_heatmap(cosine_sim_by_neighbourhood,
           x='neighbourhood', y='cosine_similarity',
           color_continuous_scale=PALETTE)

In [42]:
df_texts.columns

Index(['date', 'listing_id_encod', 'listing_id', 'reviewer_id',
       'reviewer_name', 'comments', 'host_id', 'host_name', 'host_description',
       'neighbourhood', 'latitude', 'longitude', 'room_type', 'price',
       'number_of_reviews', 'last_review', 'reviews_per_month',
       'number_of_reviews_ltm', 'review_scores_rating',
       'review_scores_accuracy', 'review_scores_cleanliness',
       'review_scores_checkin', 'review_scores_communication',
       'review_scores_location', 'review_scores_value', 'comments_language',
       'description_language', 'dominant_topic', 'percent_contribution',
       'topic_keywords', 'text', 'jaccard_similarity', 'cosine_similarity'],
      dtype='object')

In [43]:
px.imshow(df_texts[['price', 'number_of_reviews', 
                           'reviews_per_month', 'number_of_reviews_ltm', 
                           'review_scores_rating', 'jaccard_similarity', 'cosine_similarity']].corr())